In [ ]:
# url = https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib

%matplotlib inline

In [ ]:
plt.style.use('ggplot')
from matplotlib.pyplot import figure

matplotlib.rcParams['figure.figsize'] = (12, 8)
pd.options.mode.chained_assignment = None

In [ ]:
df = pd.read_csv('../Data/数据清洗.csv')

print(df.shape)
print(df.dtypes)

df_non_numeric = df.select_dtypes(include=[np.number])
non_numeric_col = df_non_numeric.columns.values
print(non_numeric_col)

In [ ]:
cols = df.columns[:30]
colours = ['#000099','#ffff00']
sns.heatmap(df[cols].isnull(),cmap=sns.color_palette(colours))

In [ ]:
for col in df.columns:
    pct_missing =np.mean(df[col].isnull())
    print('{} - {}%'.format(col,round(pct_missing*100)))


In [ ]:
for col in df.columns:
    missing = df[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:
        print('created missing indicator for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
ismissing_cols = [col for col in df.columns if 'ismissing' in col]
df['num_missing'] = df[ismissing_cols].sum(axis=1)

df['num_missing'].value_counts().reset_index().sort_values(by='index').plot.bar(x='index',y='num_missing')

In [ ]:
#drop rows with a lot of missing values
ind_missing = df[df['num_missing']>35].index
df_less_missing_rows = df.drop(ind_missing,axis=0)

In [ ]:
# hospital_beds_raion has a lot of missing
# If we want to drop
cols_to_drop = ['hospital_beds_raion']
df_less_hos_beds_raion = df.drop(cols_to_drop,axis=1)

In [ ]:
# replace missing values with the median.
med = df['life_sq'].median()
print(med)
df['life_sq']=df['life_sq'].fillna(med)

In [ ]:
# impute the missing values and create the missing value indicator variables for each numeric col
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:
        print('imputing missing values for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
        med = df[col].median()
        df[col] = df[col].fillna(med)


In [ ]:
#替换缺失的数据
# categorical
df['sub_area'] = df['sub_area'].fillna('_MISSING_')

#numeric
df['life_sq'] = df['life_sq'].fillna(-999)

In [ ]:
#不规则的数据（异常值）
#直方图和方框图
#histogram of life_sq
df['life_sq'].hist(bins=100)

In [ ]:
# box plot
df.boxplot(column=['life_sq'])

In [ ]:
# 描述性统计数据
df['life_sq'].describe()

In [ ]:
#条形图
# bar chart - distribution of a categorical variable
df['ecology'].value_counts().plot.bar()

In [ ]:
#无信息或者重复值
num_rows = len(df.index)
low_information_cols = []

for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]

    if top_pct>0.95:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col,top_pct*100))
        print(cnts)
        print()

In [ ]:
#重复数据
df_dedupped = df.drop('id',axis=1).drop_duplicates()

#there were duplicate rows
print(df.shape)
print(df_dedupped.shape)

In [ ]:
key = ['timestamp','full_sq','life_sq','floor','build_year','num_room','price_doc']

df.fillna(-999).groupby(key)['id'].count().sort_values(ascending=False
                                                       ).head(20)

In [ ]:
df_dedupped2 = df.drop_duplicates(subset=key)

print(df.shape)
print(df_dedupped2.shape)

In [ ]:
df['sub_area'].value_counts(dropna=False)

In [ ]:
df['sub_area_lower'] = df['sub_area'].str.lower()
df['sub_area_lower'].value_counts(dropna=False)

In [ ]:
df_city_ex = pd.DataFrame(data={'city:[]'})

In [ ]:
df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')
df['year'] = df['timestamp_dt'].dt.year
df['month'] = df['timestamp_dt'].dt.month
df['weekday'] = df['timestamp_dt'].dt.weekday

print(df['year'].value_counts(dropna=False))
print()
print(df['month'].value_counts(dropna=False))

In [ ]:
from nltk.metrics import edit_distance

df_city_ex = pd.DataFrame(data={'city': ['torontoo', 'toronto', 'tronto', 'vancouver', 'vancover', 'vancouvr', 'montreal', 'calgary']})

df_city_ex['city_distance_toronto'] = df_city_ex['city'].map(lambda x: edit_distance(x, 'toronto'))
df_city_ex['city_distance_vancouver'] = df_city_ex['city'].map(lambda x: edit_distance(x, 'vancouver'))
df_city_ex

In [ ]:
msk = df_city_ex['city_distance_toronto'] <= 2
df_city_ex.loc[msk, 'city'] = 'toronto'

msk = df_city_ex['city_distance_vancouver'] <= 2
df_city_ex.loc[msk, 'city'] = 'vancouver'

df_city_ex

In [ ]:
# no address column in the housing dataset. So create one to show the code.
df_add_ex = pd.DataFrame(['123 MAIN St Apartment 15', '123 Main Street Apt 12   ', '543 FirSt Av', '  876 FIRst Ave.'], columns=['address'])
df_add_ex

In [ ]:
df_add_ex['address_std'] = df_add_ex['address'].str.lower()
df_add_ex['address_std'] = df_add_ex['address_std'].str.strip() # remove leading and trailing whitespace.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\.', '') # remove period.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\bstreet\\b', 'st') # replace street with st.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\bapartment\\b', 'apt') # replace apartment with apt.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\bav\\b', 'ave') # replace apartment with apt.

df_add_ex